# Importation des Package

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib
import bs4
import re
import copy
import collections as co
import plotly
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np


# Webscrapping du top 10 netflix journalier

#### Fonction qui récupère le dataframe quand on lui donne la date sous forme string "aaaa/mm/jj", url est un argument conditionnel.

In [3]:
def recup_top_netflix_gen(date,url="https://www.the-numbers.com/home-market/netflix-daily-chart/"):
    
    url_enrichi = url+date
    from urllib.request import Request, urlopen
    req_text = urlopen(Request(url_enrichi, headers={'User-Agent': 'Mozilla/5.0'})).read()
    page = bs4.BeautifulSoup(req_text, "lxml")
    #print (page)

    """On récupère les données------------------"""
    #tableau=page.find("table",{'class' : 'chart full-width'})
    tableau=page.find("div",{'id':'page_filling_chart'})
    #print(tableau)
    #table_body=tableau.find('tbody')
    #print(table_body)
    rows = tableau.find_all('tr')
    #print (rows)

  
    """On récupére tous les élements correspondant au titre et à la note et On crée un dictionnaire dans lequel on met le contenu des lignes nettoyées
    L'index est le titre, la valeur la note.
    --------------"""
    new_rows=[]#on crée une liste vide dans laquelle on va mettre le contenu des lignes
    new_cols = 10*[[]]
    dico_top_netflix = dict()
    i = 0
    for row in rows:
        cols = row.find_all('td') # c'est une liste
        cols = [ele.text.strip() for ele in cols]
        #print (cols)
        if len(cols)> 0:
            dico_top_netflix[cols[0]] = cols[1:]
    #print(dico_top_netflix)    

    """"""

    """On crée ENFIN le dataframe"""
    df_top_netflix_gen = pd.DataFrame.from_dict(dico_top_netflix,orient='index')

    """On rajoute une index différente du titre"""
    for row in rows:
        cols = row.find_all('th')
        #print(cols)
        if len(cols) > 0 : 
            cols = [ele.get_text(separator=' ').strip().title() for ele in cols]
            columns_title = cols
    #print (columns_title)
    

    """On change le nom des colonnes"""
    df_top_netflix_gen.columns=columns_title[1:]
    
    """On vire les colonnes qui ne nous intéresse pas"""
    del(df_top_netflix_gen['Yd'])
    del(df_top_netflix_gen['Lw'])
    del(df_top_netflix_gen['Watch Now'])
    
    
    """On rajoute une colonne 'Measure Date' """
    df_top_netflix_gen['Measure Date']= [date]*df_top_netflix_gen.shape[0]
    #print ("Graph au "+ date)
    return (df_top_netflix_gen)

## Concaténer les tableaux

Liste de toutes les dates valides

In [4]:
liste_dates_valides=[]
for m in range (3,13):
    m=str(m)
    if len(m)==1:
        m = "0"+m
    for d in range (1,32):
        d=str(d)
        if len(d)==1:
            d="0"+d
        date = "2020/"+m+"/"+d
        liste_dates_valides.append(date)

#On supprime les dates sans valeurs
for i in ['2020/03/01', '2020/03/02', '2020/03/03', '2020/03/04', '2020/03/05', '2020/03/06', '2020/03/07', '2020/03/08', 
          '2020/03/09', '2020/03/10', '2020/03/11', '2020/03/12', '2020/03/13', '2020/03/14', '2020/03/15', '2020/03/16', 
          '2020/03/17', '2020/03/18', '2020/03/19', '2020/03/20', '2020/03/21', '2020/03/22', '2020/03/23', '2020/03/28', 
          '2020/03/29', '2020/04/04', '2020/04/05', '2020/04/11', '2020/04/12', '2020/04/18', '2020/04/19', '2020/04/25', 
          '2020/04/26', '2020/08/09', '2020/08/30', '2020/12/10', '2020/12/11', '2020/12/12', '2020/12/13', '2020/12/14',
          '2020/12/15', '2020/12/16', '2020/12/17', '2020/12/18', '2020/12/19', '2020/12/20', '2020/12/21', '2020/12/22', 
          '2020/12/23', '2020/12/24', '2020/12/25', '2020/12/26', '2020/12/27', '2020/12/28', '2020/12/29', '2020/12/30', 
          '2020/12/31', '2020/04/31', '2020/06/31', '2020/09/31', '2020/11/31', '2020/08/06']:
    liste_dates_valides.remove(i)


#### essai

In [5]:
df_06_05=recup_top_netflix_gen("2020/06/05")
df_06_06=recup_top_netflix_gen("2020/06/06")
DF=pd.concat([df_06_05, df_06_06])
DF.sort_values('Measure Date',ascending=False)


,Title,Type,Netflix Excl.,Netflix Release Date,Days In Top 10,Viewer- Ship Score,Measure Date
8,Clueless,Movie,,,4,11,2020/06/06
7,The Help,Movie,,,4,17,2020/06/06
6,Sweet Magnolias,TV Show,Yes,"May 19, 2020",18,142,2020/06/06
5,Avatar the Last Airbender,TV Show,,,19,145,2020/06/06
4,Cloudy with a Chance of Mea…,Movie,,,2,7,2020/06/06
3,The Last Days of American C…,Movie,Yes,"Jun 5, 2020",1,6,2020/06/06
2,Fuller House,TV Show,Yes,,4,31,2020/06/06
1,Jeffrey Epstein: Filthy Rich,TV Show,Yes,"May 27, 2020",10,91,2020/06/06
2,Jeffrey Epstein: Filthy Rich,TV Show,Yes,"May 27, 2020",9,83,2020/06/05
1,Space Force,TV Show,Yes,"May 29, 2020",7,70,2020/06/05


In [6]:
mars=[]
avril=[]
mai=[]
juin=[]
juillet=[]
aout=[]
septembre=[]
octobre=[]
novembre=[]
decembre=[]
for i in liste_dates_valides:
    if i <"2020/04/01":
        mars.append(i)
for i in liste_dates_valides:
    if i < "2020/05/01" and i > "2020/03/31":
        avril.append(i)
for i in liste_dates_valides:
    if i < "2020/06/01" and i > "2020/04/31":
        mai.append(i)
for i in liste_dates_valides:
    if i < "2020/07/01" and i > "2020/05/31":
        juin.append(i)
for i in liste_dates_valides:
    if i < "2020/08/01" and i > "2020/06/31":
        juillet.append(i)
for i in liste_dates_valides:
    if i < "2020/09/01" and i > "2020/07/31":
        aout.append(i)
for i in liste_dates_valides:
    if i < "2020/10/01" and i > "2020/08/31":
        septembre.append(i)
for i in liste_dates_valides:
    if i < "2020/11/01" and i > "2020/09/31":
        octobre.append(i)
for i in liste_dates_valides:
    if i < "2020/12/01" and i > "2020/10/31":
        novembre.append(i)
for i in liste_dates_valides:
    if i > "2020/11/31":
        decembre.append(i)
mois = [mars,avril,mai,juin,juillet,aout,septembre,octobre,novembre,decembre]
mois

[['2020/03/24',
  '2020/03/25',
  '2020/03/26',
  '2020/03/27',
  '2020/03/30',
  '2020/03/31'],
 ['2020/04/01',
  '2020/04/02',
  '2020/04/03',
  '2020/04/06',
  '2020/04/07',
  '2020/04/08',
  '2020/04/09',
  '2020/04/10',
  '2020/04/13',
  '2020/04/14',
  '2020/04/15',
  '2020/04/16',
  '2020/04/17',
  '2020/04/20',
  '2020/04/21',
  '2020/04/22',
  '2020/04/23',
  '2020/04/24',
  '2020/04/27',
  '2020/04/28',
  '2020/04/29',
  '2020/04/30'],
 ['2020/05/01',
  '2020/05/02',
  '2020/05/03',
  '2020/05/04',
  '2020/05/05',
  '2020/05/06',
  '2020/05/07',
  '2020/05/08',
  '2020/05/09',
  '2020/05/10',
  '2020/05/11',
  '2020/05/12',
  '2020/05/13',
  '2020/05/14',
  '2020/05/15',
  '2020/05/16',
  '2020/05/17',
  '2020/05/18',
  '2020/05/19',
  '2020/05/20',
  '2020/05/21',
  '2020/05/22',
  '2020/05/23',
  '2020/05/24',
  '2020/05/25',
  '2020/05/26',
  '2020/05/27',
  '2020/05/28',
  '2020/05/29',
  '2020/05/30',
  '2020/05/31'],
 ['2020/06/01',
  '2020/06/02',
  '2020/06/03',
  '20

In [7]:
df_mars = recup_top_netflix_gen("2020/03/24")
for i in mars[1:]:
    a_conc = recup_top_netflix_gen(i)
    df_mars = pd.concat([df_mars,a_conc])

In [8]:
df_avril = recup_top_netflix_gen("2020/04/01")
a_conc=0
for i in avril[1:]:
    a_conc = recup_top_netflix_gen(i)
    df_avril = pd.concat([df_avril,a_conc])

In [9]:
df_mai = recup_top_netflix_gen("2020/05/01")
a_conc=0
for i in mai[1:]:
    a_conc = recup_top_netflix_gen(i)
    df_mai = pd.concat([df_mai,a_conc])

In [12]:
df_juin = recup_top_netflix_gen("2020/06/01")
a_conc=0
for i in juin[1:]:
    a_conc = recup_top_netflix_gen(i)
    df_juin = pd.concat([df_juin,a_conc])

In [13]:
df_juillet = recup_top_netflix_gen("2020/07/01")
a_conc=0
for i in juillet[1:]:
    a_conc = recup_top_netflix_gen(i)
    df_juillet = pd.concat([df_juillet,a_conc])

In [14]:
df_aout = recup_top_netflix_gen("2020/08/01")
a_conc=0
for i in aout[1:]:
    a_conc = recup_top_netflix_gen(i)
    df_aout = pd.concat([df_aout,a_conc])

In [17]:
df_septembre = recup_top_netflix_gen("2020/09/01")
a_conc=0
for i in septembre[1:]:
    a_conc = recup_top_netflix_gen(i)
    df_septembre = pd.concat([df_septembre,a_conc])

In [18]:
df_octobre = recup_top_netflix_gen("2020/10/01")
a_conc=0
for i in octobre[1:]:
    a_conc = recup_top_netflix_gen(i)
    df_octobre = pd.concat([df_octobre,a_conc])

In [19]:
df_novembre = recup_top_netflix_gen("2020/11/01")
a_conc=0
for i in novembre[1:]:
    a_conc = recup_top_netflix_gen(i)
    df_novembre = pd.concat([df_novembre,a_conc])

In [20]:
df_decembre = recup_top_netflix_gen("2020/11/01")
a_conc=0
for i in decembre[1:]:
    a_conc = recup_top_netflix_gen(i)
    df_decembre = pd.concat([df_decembre,a_conc])

In [ ]:
dico_df_mois={"mars" : df_mars, "avril" : df_avril, "mai":df_mai , "juin":df_juin , "juillet":df_juillet,
              "aout":df_aout ,"septembre" :df_septembre ,"octobre": df_octobre, "novembre" :df_novembre,"decembre": df_decembre}

for moi, df_moi in dico_df_mois.items():
    a=df_moi.to_csv("df_"+moi, index = True) 